##### Аўтарскае права 2020 Аўтары TensorFlow.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Пашыраная аўтаматычная дыферэнцыяцыя

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/advanced_autodiff"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Праглядзець на TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Запусціце ў Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Паглядзець крыніцу на GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Спампаваць сшытак</a></td>
</table>

[Аўтаматычнае кіраўніцтва па дыферэнцыяцыі](autodiff.ipynb) ўключае ў сябе ўсё неабходнае для разліку градыентаў. У гэтым кіраўніцтве прысвечана больш глыбокім, менш распаўсюджаным асаблівасцям API `tf.GradientTape` .

## Усталяваць

In [ ]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.figsize'] = (8, 6)

## Кіраванне градыентнай запісам

У кіраўніцтве па [аўтаматычнай дыферэнцыяцыі](autodiff.ipynb) вы ўбачылі, як кантраляваць, якія зменныя і тэнзары назіраюцца на стужцы пры стварэнні разліку градыенту.

Стужка таксама мае метады маніпулявання запісам.

Калі вы хочаце спыніць запіс градыентаў, вы можаце выкарыстоўваць `GradientTape.stop_recording()` каб часова прыпыніць запіс.

Гэта можа быць карысна для памяншэння накладных выдаткаў, калі вы не хочаце адрозніваць складаную аперацыю ў сярэдзіне вашай мадэлі. Гэта можа ўключаць разлік метрыкі альбо прамежкавы вынік:

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as t:
  x_sq = x * x
  with t.stop_recording():
    y_sq = y * y
  z = x_sq + y_sq

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])

Калі вы хочаце пачаць усё спачатку, выкарыстоўвайце `reset()` . Просты выхад з блока градыентнай стужкі і перазапуск звычайна лягчэй чытаць, але вы можаце выкарыстоўваць `reset` калі выхад з блока стужкі складаны альбо немагчымы.

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)
reset = True

with tf.GradientTape() as t:
  y_sq = y * y
  if reset:
    # Throw out all the tape recorded so far
    t.reset()
  z = x * x + y_sq

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])

## Стоп-градыент

У адрозненне ад глабальных элементаў кіравання стужкай вышэй, функцыя `tf.stop_gradient` значна больш дакладная. З яго дапамогай можна спыніць пераход градыентаў па пэўным шляху без неабходнасці доступу да самой стужкі:

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as t:
  y_sq = y**2
  z = x**2 + tf.stop_gradient(y_sq)

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])

## Нестандартныя градыенты

У некаторых выпадках вы можаце кантраляваць, як менавіта вылічваюцца градыенты, а не выкарыстоўваць стандартную. Гэтыя сітуацыі ўключаюць:

- Для новай працы, якую вы пішаце, не вызначаны градыент.
- Разлікі па змаўчанні няўстойлівыя.
- Вы хочаце кэшаваць дарагія вылічэнні з праходжання.
- Вы хочаце змяніць значэнне (напрыклад, выкарыстоўваючы: `tf.clip_by_value` , `tf.math.round` ) без змены градыенту.

Для напісання новай працы вы можаце выкарыстоўваць `tf.RegisterGradient` каб стварыць уласную. Падрабязнасці глядзіце на гэтай старонцы. (Звярніце ўвагу, што рэестр градыентаў з'яўляецца глабальным, таму змяняйце яго з асцярожнасцю.)

У апошніх трох выпадках вы можаце выкарыстоўваць `tf.custom_gradient` .


Вось прыклад, які прымяняе `tf.clip_by_norm` да прамежкавага градыенту.

In [ ]:
# Establish an identity operation, but clip during the gradient pass
@tf.custom_gradient
def clip_gradients(y):
  def backward(dy):
    return tf.clip_by_norm(dy, 0.5)
  return y, backward

v = tf.Variable(2.0)
with tf.GradientTape() as t:
  output = clip_gradients(v * v)
print(t.gradient(output, v))  # calls "backward", which clips 4 to 2


See the `tf.custom_gradient` decorator for more details.

## Некалькі стужак

Некалькі стужак бесперашкодна ўзаемадзейнічаюць. Напрыклад, тут кожная стужка глядзіць розныя наборы тэнзараў:

In [ ]:
x0 = tf.constant(0.0)
x1 = tf.constant(0.0)

with tf.GradientTape() as tape0, tf.GradientTape() as tape1:
  tape0.watch(x0)
  tape1.watch(x1)

  y0 = tf.math.sin(x0)
  y1 = tf.nn.sigmoid(x1)

  y = y0 + y1

  ys = tf.reduce_sum(y)

In [ ]:
tape0.gradient(ys, x0).numpy()   # cos(x) => 1.0

In [ ]:
tape1.gradient(ys, x1).numpy()   # sigmoid(x1)*(1-sigmoid(x1)) => 0.25

### Градыенты вышэйшага парадку

Аперацыі ўнутры дыспетчара кантэксту `GradientTape` запісваюцца для аўтаматычнай дыферэнцыяцыі. Калі градыенты вылічаюцца ў гэтым кантэксце, то вылічэнне градыенту таксама запісваецца. У выніку, сапраўды такі ж API працуе і для градыентаў вышэйшага парадку. Напрыклад:

In [ ]:
x = tf.Variable(1.0)  # Create a Tensorflow variable initialized to 1.0

with tf.GradientTape() as t2:
  with tf.GradientTape() as t1:
    y = x * x * x

  # Compute the gradient inside the outer `t2` context manager
  # which means the gradient computation is differentiable as well.
  dy_dx = t1.gradient(y, x)
d2y_dx2 = t2.gradient(dy_dx, x)

print('dy_dx:', dy_dx.numpy())  # 3 * x**2 => 3.0
print('d2y_dx2:', d2y_dx2.numpy())  # 6 * x => 6.0

Хоць гэта і дае вам другую вытворную *скалярнай* функцыі, гэтая мадэль не абагульняе, ствараючы матрыцу Гесія, бо `GradientTape.gradient` вылічвае толькі градыент скаляра. Каб пабудаваць гесія, гл. [Гесэнскі прыклад](#hessian) у [раздзеле Якабіян](#jacobians) .

"Укладзеныя выклікі `GradientTape.gradient` " - гэта добры ўзор, калі вы вылічаеце скаляр па градыенце, а затым атрыманы скаляр выступае ў якасці крыніцы для разліку другога градыенту, як у наступным прыкладзе.


#### Прыклад: рэгулярызацыя ўваходнага градыенту

Шмат якія мадэлі ўспрымальныя да "супярэчлівых прыкладаў". Гэтая калекцыя метадаў змяняе ўвод мадэлі, каб заблытаць выхад мадэлі. Самая [простая рэалізацыя](https://www.tensorflow.org/tutorials/generative/adversarial_fgsm) займае адзін крок па градыенце вываду адносна ўводу; "градыент уводу".

Адным з метадаў павышэння надзейнасці спаборніцкіх прыкладаў з'яўляецца [рэгулярызацыя ўваходнага градыенту](https://arxiv.org/abs/1905.11468) , якая спрабуе мінімізаваць велічыню ўваходнага градыента. Калі ўваходны градыент невялікі, то і змяненне выходных дадзеных павінна быць невялікім.

Ніжэй прыведзена наіўная рэалізацыя рэгулярызацыі ўваходных градыентаў. Рэалізацыя:

1. Вылічыце градыент выходных дадзеных адносна ўваходных дадзеных, выкарыстоўваючы ўнутраную стужку.
2. Вылічыце велічыню гэтага ўваходнага градыенту.
3. Разлічыце градыент гэтай велічыні адносна мадэлі.

In [ ]:
x = tf.random.normal([7, 5])

layer = tf.keras.layers.Dense(10, activation=tf.nn.relu)

In [ ]:
with tf.GradientTape() as t2:
  # The inner tape only takes the gradient with respect to the input,
  # not the variables.
  with tf.GradientTape(watch_accessed_variables=False) as t1:
    t1.watch(x)
    y = layer(x)
    out = tf.reduce_sum(layer(x)**2)
  # 1. Calculate the input gradient.
  g1 = t1.gradient(out, x)
  # 2. Calculate the magnitude of the input gradient.
  g1_mag = tf.norm(g1)

# 3. Calculate the gradient of the magnitude with respect to the model.
dg1_mag = t2.gradient(g1_mag, layer.trainable_variables)

In [ ]:
[var.shape for var in dg1_mag]

## Якабіянцы


Усе папярэднія прыклады прымалі градыенты скалярнай мэты ў адносінах да нейкага крыніцы тэнзара.

[Матрыца Якабіяна](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant) ўяўляе градыенты вектарнай функцыі. Кожны радок утрымлівае градыент аднаго з элементаў вектара.

Метад `GradientTape.jacobian` дазваляе эфектыўна вылічыць матрыцу Якабіяна.

Звярніце ўвагу, што:

- Як `gradient` : Аргументам `sources` можа быць тэнсар альбо кантэйнер тэнзараў.
- У адрозненне ад `gradient` : `target` тэнзар павінен быць адзінкавым.

### Скалярная крыніца

У якасці першага прыкладу, вось якабіян вектара-мішэні адносна скалярнай крыніцы.

In [ ]:
x = tf.linspace(-10.0, 10.0, 200+1)
delta = tf.Variable(0.0)

with tf.GradientTape() as tape:
  y = tf.nn.sigmoid(x+delta)

dy_dx = tape.jacobian(y, delta)

Калі вы бераце якабіян у адносінах да скаляра, вынік мае форму **мэты** і дае градыент кожнага элемента ў адносінах да крыніцы:

In [ ]:
print(y.shape)
print(dy_dx.shape)

In [ ]:
plt.plot(x.numpy(), y, label='y')
plt.plot(x.numpy(), dy_dx, label='dy/dx')
plt.legend()
_ = plt.xlabel('x')

### Крыніца тэнзара

Незалежна ад таго, ці з'яўляецца ўвод скалярным ці тэнзарным, `GradientTape.jacobian` эфектыўна вылічвае градыент кожнага элемента крыніцы ў адносінах да кожнага элемента мэты.

Напрыклад, выхад гэтага пласта мае форму `(10, 7)` :

In [ ]:
x = tf.random.normal([7, 5])
layer = tf.keras.layers.Dense(10, activation=tf.nn.relu)

with tf.GradientTape(persistent=True) as tape:
  y = layer(x)

y.shape

І форма ядра пласта `(5, 10)` :

In [ ]:
layer.kernel.shape

Форма якабія вываду адносна ядра - гэта тыя дзве формы, аб'яднаныя разам:

In [ ]:
j = tape.jacobian(y, layer.kernel)
j.shape

Калі сумаваць па памерах мэты, вам застаецца градыент сумы, які быў бы вылічаны `GradientTape.gradient` :

In [ ]:
g = tape.gradient(y, layer.kernel)
print('g.shape:', g.shape)

j_sum = tf.reduce_sum(j, axis=[0, 1])
delta = tf.reduce_max(abs(g - j_sum)).numpy()
assert delta < 1e-3
print('delta:', delta)

<a id="hessian"> </a>

#### Прыклад: гесенская

While `tf.GradientTape` doesn't give an explicit method for constructing a Hessian matrix it's possible to build one using the `GradientTape.jacobian` method.

Заўвага: матрыца Гесія ўтрымлівае параметры `N**2` . Па гэтай і іншых прычынах гэта не практычна для большасці мадэляў. Гэты прыклад уключаны больш як дэманстрацыя таго, як выкарыстоўваць метад `GradientTape.jacobian` , і не з'яўляецца рэкамендацыяй прамой аптымізацыі на аснове Гесія. Даведнік Гесія-вектара можна [эфектыўна вылічыць з укладзенымі стужкамі](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/eager/benchmarks/resnet50/hvp_test.py) і з'яўляецца значна больш эфектыўным падыходам да аптымізацыі другога парадку.


In [ ]:
x = tf.random.normal([7, 5])
layer1 = tf.keras.layers.Dense(8, activation=tf.nn.relu)
layer2 = tf.keras.layers.Dense(6, activation=tf.nn.relu)

with tf.GradientTape() as t2:
  with tf.GradientTape() as t1:
    x = layer1(x)
    x = layer2(x)
    loss = tf.reduce_mean(x**2)

  g = t1.gradient(loss, layer1.kernel)

h = t2.jacobian(g, layer1.kernel)

In [ ]:
print(f'layer.kernel.shape: {layer1.kernel.shape}')
print(f'h.shape: {h.shape}')

Каб выкарыстаць гэты гесіян для кроку метаду Ньютана, спачатку трэба выраўнаваць яго восі ў матрыцу, а градыент - у вектар:

In [ ]:
n_params = tf.reduce_prod(layer1.kernel.shape)

g_vec = tf.reshape(g, [n_params, 1])
h_mat = tf.reshape(h, [n_params, n_params])

Матрыца Гесія павінна быць сіметрычнай:

In [ ]:
def imshow_zero_center(image, **kwargs):
  lim = tf.reduce_max(abs(image))
  plt.imshow(image, vmin=-lim, vmax=lim, cmap='seismic', **kwargs)
  plt.colorbar()

In [ ]:
imshow_zero_center(h_mat)

Крок абнаўлення метаду Ньютана паказаны ніжэй.

In [ ]:
eps = 1e-3
eye_eps = tf.eye(h_mat.shape[0])*eps

Заўвага: На [самай справе не інвертуйце матрыцу](https://www.johndcook.com/blog/2010/01/19/dont-invert-that-matrix/) .

In [ ]:
# X(k+1) = X(k) - (∇²f(X(k)))^-1 @ ∇f(X(k))
# h_mat = ∇²f(X(k))
# g_vec = ∇f(X(k))
update = tf.linalg.solve(h_mat + eye_eps, g_vec)

# Reshape the update and apply it to the variable.
_ = layer1.kernel.assign_sub(tf.reshape(update, layer1.kernel.shape))

Хоць гэта адносна проста для адной `tf.Variable` , прымяненне гэтага да нетрывіяльнай мадэлі запатрабуе ўважлівага аб'яднання і нарэзкі для атрымання поўнага гесія па некалькіх зменных.

### Партыя якабіян

У некаторых выпадках вы хочаце ўзяць якабіянаў з кожнага стэка мішэняў адносна стэка крыніц, дзе якабінцы для кожнай пары мэта-крыніца незалежныя.

Напрыклад, тут уваходны `x` мае форму `(batch, ins)` а выхадны `y` - `(batch, outs)` .


In [ ]:
x = tf.random.normal([7, 5])

layer1 = tf.keras.layers.Dense(8, activation=tf.nn.elu)
layer2 = tf.keras.layers.Dense(6, activation=tf.nn.elu)

with tf.GradientTape(persistent=True, watch_accessed_variables=False) as tape:
  tape.watch(x)
  y = layer1(x)
  y = layer2(y)

y.shape

Поўны якабіян `y` па адносінах да `x` мае форму `(batch, ins, batch, outs)` , нават калі вы хочаце толькі `(batch, ins, outs)` .

In [ ]:
j = tape.jacobian(y, x)
j.shape

Калі градыенты кожнага элемента ў стэку незалежныя, то кожны `(batch, batch)` зрэз гэтага тэнзара з'яўляецца дыяганальнай матрыцай:

In [ ]:
imshow_zero_center(j[:, 0, :, 0])
_ = plt.title('A (batch, batch) slice')

In [ ]:
def plot_as_patches(j):
  # Reorder axes so the diagonals will each form a contiguous patch.
  j = tf.transpose(j, [1, 0, 3, 2])
  # Pad in between each patch.
  lim = tf.reduce_max(abs(j))
  j = tf.pad(j, [[0, 0], [1, 1], [0, 0], [1, 1]],
             constant_values=-lim)
  # Reshape to form a single image.
  s = j.shape
  j = tf.reshape(j, [s[0]*s[1], s[2]*s[3]])
  imshow_zero_center(j, extent=[-0.5, s[2]-0.5, s[0]-0.5, -0.5])

plot_as_patches(j)
_ = plt.title('All (batch, batch) slices are diagonal')

Каб атрымаць жаданы вынік, вы можаце падсумаваць памер дубліката `batch` , альбо выберыце дыяганалі, выкарыстоўваючы `tf.einsum` .


In [ ]:
j_sum = tf.reduce_sum(j, axis=2)
print(j_sum.shape)
j_select = tf.einsum('bxby->bxy', j)
print(j_select.shape)

Было б значна больш эфектыўна рабіць разлік без лішніх памераў. Метад `GradientTape.batch_jacobian` робіць менавіта гэта.

In [ ]:
jb = tape.batch_jacobian(y, x)
jb.shape

In [ ]:
error = tf.reduce_max(abs(jb - j_sum))
assert error < 1e-3
print(error.numpy())

Caution: `GradientTape.batch_jacobian` only verifies that the first dimension of the source and target match. It doesn't check that the gradients are actually independent. It's up to the user to ensure they only use `batch_jacobian` where it makes sense. For example adding a `layers.BatchNormalization` destroys the independence, since it normalizes across the `batch` dimension:

In [ ]:
x = tf.random.normal([7, 5])

layer1 = tf.keras.layers.Dense(8, activation=tf.nn.elu)
bn = tf.keras.layers.BatchNormalization()
layer2 = tf.keras.layers.Dense(6, activation=tf.nn.elu)

with tf.GradientTape(persistent=True, watch_accessed_variables=False) as tape:
  tape.watch(x)
  y = layer1(x)
  y = bn(y, training=True)
  y = layer2(y)

j = tape.jacobian(y, x)
print(f'j.shape: {j.shape}')

In [ ]:
plot_as_patches(j)

_ = plt.title('These slices are not diagonal')
_ = plt.xlabel("Don't use `batch_jacobian`")

У гэтым выпадку `batch_jacobian` ранейшаму працуе і вяртае *нешта* з чаканай формай, але яго змест мае незразумелы сэнс.

In [ ]:
jb = tape.batch_jacobian(y, x)
print(f'jb.shape: {jb.shape}')